In [1]:
import os

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

load_dotenv()

# llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(model=os.getenv("DEFAULT_MODEL"))

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='저는 다양한 질문에 답하고 정보를 제공할 수 있는 언어 모델입니다. 제가 하는 일은 매일 매일 개선하고 있지만, 감정이나 감각을 느끼는 능력은 없다는 사실을 알고 계셨나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 15, 'total_tokens': 58, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.346077722, 'prompt_time': 0.002881445, 'completion_time': 0.109240112, 'total_time': 0.112121557}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'id': 'chatcmpl-dcbeb10b-eb41-487b-acd3-c91cf16d03f2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--402faf6c-b35a-432f-9180-2b79127c9366-0', usage_metadata={'input_tokens': 15, 'output_tokens': 43, 'total_tokens': 58, 'input_token_details': {}, 'output_token_details': {}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [3]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("""너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.
    주식 가격 정보를 요청받을 때는 반드시 제공된 get_yf_stock_history 함수를 사용해야 한다.
    직접 데이터를 생성하지 말고, 항상 제공된 도구를 통해 데이터를 조회하라."""),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.\n    주식 가격 정보를 요청받을 때는 반드시 제공된 get_yf_stock_history 함수를 사용해야 한다.\n    직접 데이터를 생성하지 말고, 항상 제공된 도구를 통해 데이터를 조회하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'w1ae2xk3q', 'function': {'arguments': '{"location":"부산","timezone":"Asia/Seoul"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 793, 'total_tokens': 833, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.232249716, 'prompt_time': 0.032127791, 'completion_time': 0.091374234, 'total_time': 0.123502025}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'id': 'chatcmpl-425f7062-ca5a-4b4f-bdcc-2b290915fe5e', 'service_tier': 'on_demand', 'finish_reason'

##### request, response body
```text
REQUEST >> https://api.fireworks.ai/inference/v1/chat/completions
{
  "messages": [
    {
      "content": "너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.",
      "role": "system"
    },
    {
      "content": "부산은 지금 몇시야?",
      "role": "user"
    }
  ],
  "model": "accounts/fireworks/models/llama4-maverick-instruct-basic",
  "stream": false,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "get_current_time",
        "description": "현재 시각을 반환하는 함수\n\n   Args:\n       timezone (str): 타임존 (예: \u0027Asia/Seoul\u0027) 실제 존재하는 타임존이어야 함\n       location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨",
        "parameters": {
          "properties": {
            "timezone": {
              "type": "string"
            },
            "location": {
              "type": "string"
            }
          },
          "required": [
            "timezone",
            "location"
          ],
          "type": "object"
        }
      }
    }
  ]
}
RESPONSE >> 200 OK
{
  "id": "17c3eded-93e2-4cd7-b4d4-31c172097814",
  "object": "chat.completion",
  "created": 1754892171,
  "model": "accounts/fireworks/models/llama4-maverick-instruct-basic",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "tool_calls": [
          {
            "index": 0,
            "id": "call_eRp4wDrn1AR2f7UflqeUZ8X4",
            "type": "function",
            "function": {
              "name": "get_current_time",
              "arguments": "{\"timezone\": \"Asia/Seoul\", \"location\": \"Busan\"}"
            }
          }
        ]
      },
      "finish_reason": "tool_calls"
    }
  ],
  "usage": {
    "prompt_tokens": 241,
    "total_tokens": 258,
    "completion_tokens": 17
  }
}```

In [5]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'location': '부산', 'timezone': 'Asia/Seoul'}
Asia/Seoul (부산) 현재시각 2025-08-13 01:01:14 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.\n    주식 가격 정보를 요청받을 때는 반드시 제공된 get_yf_stock_history 함수를 사용해야 한다.\n    직접 데이터를 생성하지 말고, 항상 제공된 도구를 통해 데이터를 조회하라.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'w1ae2xk3q', 'function': {'arguments': '{"location":"부산","timezone":"Asia/Seoul"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 793, 'total_tokens': 833, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.232249716, 'prompt_time': 0.032127791, 'completion_time': 0.091374234, 'total_time': 0.123502025}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'id': 'chatcmpl-425f7062-ca5a-4b4f-bdcc-2b290915fe5e', 'service_tier': 'on_demand', 'finish_reaso

In [6]:
llm_with_tools.invoke(messages)

AIMessage(content='현재 부산은 01시 01분 14초 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 865, 'total_tokens': 880, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.299846838, 'prompt_time': 0.032808283, 'completion_time': 0.038597364, 'total_time': 0.071405647}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'id': 'chatcmpl-1c2bd7e3-1aa0-41c7-9ae1-88ba4133b2cd', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec025a74-2854-4a21-80da-6206944a4c54-0', usage_metadata={'input_tokens': 865, 'output_tokens': 15, 'total_tokens': 880, 'input_token_details': {}, 'output_token_details': {}})

In [7]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [8]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [9]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

# NOTE: 이 책의 의도상, 이 시점에서 llm 은 어떤 function 을 어떤 args 로 실행 해야 할지를 알려줘야 하는데, invoke 문에서 이미 tool function 이 실행 되었음.
#       model : accounts/fireworks/models/llama4-maverick-instruct-basic
#       > test_with_tools.py
#       > LLM 이 헛소리를 하는 거 였음. llama 문제인가? > system message 에 반드시 함수를 이용하도록 강화

content='' additional_kwargs={'tool_calls': [{'id': 'scs1rj2ec', 'function': {'arguments': '{"stock_history_input":{"period":"1mo","ticker":"TSLA"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 978, 'total_tokens': 1019, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.714986359, 'prompt_time': 0.038328631, 'completion_time': 0.108980784, 'total_time': 0.147309415}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'id': 'chatcmpl-cbb0135b-5377-43f6-b6d9-e3509647c876', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--d43114f1-85cc-4a67-9ae3-4164956ced91-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'period': '1mo', 'ticker': 'TSLA'}}, 'id': 'scs1rj2ec', 'type': 'tool_call'}] usage_metadata={'input_tokens': 978, 'output_tokens': 41

In [10]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'period': '1mo', 'ticker': 'TSLA'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-07-14 00:00:00-04:00 | 317.73 | 322.6  | 312.67 |  316.9  | 7.80434e+07 |           0 |              0 |\n| 2025-07-15 00:00:00-04:00 | 319.68 | 321.2  | 310.5  |  310.78 | 7.75563e+07 |           0 |              0 |\n| 2025-07-16 00:00:00-04:00 | 312.8  | 323.5  | 312.62 |  321.67 | 9.72848e+07 |           0 |              0 |\n| 2025-07-17 00:00:00-04:00 | 323.15 | 324.34 | 317.06 |  319.41 | 7.39229e+07 |           0 |              0 |\n| 2025-07-18 00:00:00-04:00 | 321.66 | 330.9  | 321.42 |  329.65 | 9.4255e+07  |           0 |              0 |\n| 2025-07-21 00:00:00-04:00 | 334.4  | 338    | 326.88 |  328.49 | 7.57688e+07 |           0 |              0 |\n| 2025-07-22 00:00:00-04:0

In [11]:
llm_with_tools.invoke(messages)

AIMessage(content='테슬라는 한달 전에 비해 주가가 올랐다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 2055, 'total_tokens': 2067, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.929292711, 'prompt_time': 0.098889333, 'completion_time': 0.030860327, 'total_time': 0.12974966}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_79da0e0073', 'id': 'chatcmpl-4239dcf1-f178-4f26-9d1f-9c6ae55ca350', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bf175294-9a27-4e3a-b48e-3e4753b05e20-0', usage_metadata={'input_tokens': 2055, 'output_tokens': 12, 'total_tokens': 2067, 'input_token_details': {}, 'output_token_details': {}})